In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(500)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/5.vehicle'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header

import time

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

# = = = = = = = = = = = = = = =

directory_in = './file/4.link'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['Part Code'] = df_input['Part Code'].astype(int)
    df_input['Make Code'] = df_input['Make Code'].astype(int)
    df_input['Link No'] = df_input['Link No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global api, list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            data = {'getArticleLinkedAllLinkingTargetsByIds3': {'articleCountry': 'DE',
                                                                'articleId': crawler_series['Part Code'],
                                                                'immediateAttributs': True,
                                                                'lang': 'qa',
                                                                'linkedArticlePairs': json.loads(crawler_series['Json_Link']),
                                                                'linkingTargetType': 'VOLB',
                                                                'provider': 23365}}

            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_check = crawler_series['Url']

                    resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                         data=json.dumps(data),
                                         headers=get_header(**{'x-api-key': api}),
                                         timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_ = resp.json()

                        if dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                            print(f'fuck api {api}')
                            time.sleep(3)
                            with open('./api.txt', 'r') as api_txt:
                                api = api_txt.read()
                        elif dict_['status'] == 200:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            list_vehicle = dict_['data']['array']

            # = = = = = = = = = = = = = = =

            if not list_vehicle:
                raise

            # = = = = = = = = = = = = = = =

            list_dict_correct_temp = []
            for i, vehicle in enumerate(list_vehicle):
                if len(vehicle['linkedVehicles']['array']) != 1:
                    print('fuck')

                if crawler_series['Make'] != vehicle['linkedVehicles']['array'][0]['manuDesc'].strip():
                    raise

                dict_vehicle = {'Part Code': crawler_series['Part Code'],
                                'Link No': crawler_series['Link No'],
                                'Json_Link': crawler_series['Json_Link'],
                                'JOIN_PCMCLN': crawler_series['JOIN_JOIN_PCMCLN'],
                                'Vehicle No': i + 1,
                                'Make': crawler_series['Make'],
                                'Model': vehicle['linkedVehicles']['array'][0]['modelDesc'].strip(),
                                'Submodel': vehicle['linkedVehicles']['array'][0]['carDesc'].strip(),
                                'Year': f'''{vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom']}-{vehicle['linkedVehicles']['array'][0]['yearOfConstructionTo']}''' if 'yearOfConstructionTo' in vehicle['linkedVehicles']['array'][0] and vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom'] != vehicle['linkedVehicles']['array'][0]['yearOfConstructionTo'] else f'''{vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom']}-''',
                                'kW': vehicle['linkedVehicles']['array'][0]['powerKwFrom'] if vehicle['linkedVehicles']['array'][0]['powerKwFrom'] == vehicle['linkedVehicles']['array'][0]['powerKwTo'] else f'''{vehicle['linkedVehicles']['array'][0]['powerKwFrom']}-{vehicle['linkedVehicles']['array'][0]['powerKwTo']}''',
                                'HP': vehicle['linkedVehicles']['array'][0]['powerHpFrom'] if vehicle['linkedVehicles']['array'][0]['powerHpFrom'] == vehicle['linkedVehicles']['array'][0]['powerHpTo'] else f'''{vehicle['linkedVehicles']['array'][0]['powerHpFrom']}-{vehicle['linkedVehicles']['array'][0]['powerHpTo']}''',
                                'cc': vehicle['linkedVehicles']['array'][0]['cylinderCapacity'] if 'cylinderCapacity' in vehicle['linkedVehicles']['array'][0] else '',
                                'Design': vehicle['linkedVehicles']['array'][0]['constructionType'].strip()}

                if type(vehicle['linkedArticleImmediateAttributs']) == str and vehicle['linkedArticleImmediateAttributs'].strip() == '':
                    list_attribute = []
                elif type(vehicle['linkedArticleImmediateAttributs']) == dict:
                    list_attribute = vehicle['linkedArticleImmediateAttributs']['array']
                else:
                    raise

                dict_count = {}
                for attribute in list_attribute:
                    if attribute['attrName'].strip() == 'Block Separation' and attribute['attrShortName'].strip() == 'or' and 'attrValue' not in attribute:
                        continue
                    key = attribute['attrName'].strip()
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1

                dict_no = {}
                for attribute in list_attribute:
                    if 'attrValue' not in attribute:
                        continue
                    key = attribute['attrName'].strip()
                    if dict_count[key] == 1:
                        dict_vehicle[key] = attribute['attrValue'].strip()
                    else:
                        if key in dict_no:
                            dict_no[key] += 1
                        else:
                            dict_no[key] = 1
                        dict_vehicle[f'{key}-{dict_no[key]}'] = attribute['attrValue'].strip()

                list_dict_correct_temp.append(dict_vehicle)

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.extend(list_dict_correct_temp)

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['JOIN_PCMC']}. {url_check}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Code', 'Make', 'Link No', 'Vehicle No'],
                                                                 ascending=[True, True, True, True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Code', 'Make', 'Link No'],
                                                               ascending=[True, True, True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

[True] - [请求1次] - [剩余299条] - [crawler_1（50.00%）] - [2024/11/07 10:06:58] - 1225701240;5. https://web.tecalliance.net/tecdoc/qa/parts/21/566007/detail?groups=298

[True] - [请求1次] - [剩余298条] - [crawler_1（50.00%）] - [2024/11/07 10:06:58] - 1225701240;120. https://web.tecalliance.net/tecdoc/qa/parts/21/566007/detail?groups=298

[True] - [请求1次] - [剩余297条] - [crawler_1（50.00%）] - [2024/11/07 10:06:58] - 1225703574;74. https://web.tecalliance.net/tecdoc/qa/parts/21/566009/detail?groups=298

[True] - [请求1次] - [剩余296条] - [crawler_1（50.00%）] - [2024/11/07 10:06:58] - 1225703574;74. https://web.tecalliance.net/tecdoc/qa/parts/21/566009/detail?groups=298

[True] - [请求1次] - [剩余295条] - [crawler_1（50.00%）] - [2024/11/07 10:06:58] - 1225703574;84. https://web.tecalliance.net/tecdoc/qa/parts/21/566009/detail?groups=298

[True] - [请求1次] - [剩余294条] - [crawler_1（50.00%）] - [2024/11/07 10:06:58] - 1225697307;84. https://web.tecalliance.net/tecdoc/qa/parts/21/566005/detail?groups=298

[True] - [请求1次] - [剩余2

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4017.53it/s]

Done ~
